In this experiment, you will train models to distringuish examples of two different genres of Shakespeare's plays: comedies and tragedies. (We'll ignore the histories, sonnets, etc.) Since he died four hundred years ago, Shakespeare has not written any more plays—although scraps of various other works have come to light. We are not, therefore, interested in building models simply to help categorize an unbounded stream of future documents, as we might be in other applications of text classification; rather, we are interested in what a classifier might have to tell us about what we mean by the terms “comedy” and “tragedy”.

You will start by copying and running your `createBasicFeatures` function from the experiment with movie reviews. Do the features the classifier focuses on tell you much about comedy and tragedy in general?

You will then implement another featurization function `createInterestingFeatures`, which will focus on only those features you think are informative for distinguishing between comedy and tragedy. Accuracy on leave-one-out cross-validation may go up, but it more important to look at the features given the highest weight by the classifier. Interpretability in machine learning, of course, may be harder to define than accuracy—although accuracy at some tasks such as summarization is hard enoough.

In [68]:
import json
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,LeaveOneOut
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [71]:
#read in the shakespeare corpus
def readShakespeare():
  raw = requests.get("https://raw.githubusercontent.com/mutherr/CS6120-PS1-data/master/shakespeare_plays.json").text.strip()
  corpus = [json.loads(line) for line in raw.split("\n")]

  #remove histories from the data, as we're only working with tragedies and comedies
  corpus = [entry for entry in corpus if entry["genre"] != "history"]
  return corpus

This is where you will implement two functions to featurize the data:

In [72]:
#NB: The current contents are for testing only
#This function should return: 
#  -a sparse numpy matrix of document features
#  -a list of the correct genre for each document
#  -a list of the vocabulary used by the features, such that the ith term of the
#    list is the word whose counts appear in the ith column of the matrix. 

# This function should create a feature representation using all tokens that
# contain an alphabetic character.
def createBasicFeatures(corpus):
  #Your code here
  data = pd.DataFrame(corpus)
  vectorizer = CountVectorizer(
      token_pattern = '\w*[a-zA-Z]\w*',
      decode_error = 'ignore',
      analyzer = 'word'
  )
  texts = vectorizer.fit_transform(data.text)
  genres = data['genre']
  vocab = vectorizer.get_feature_names()

  
  return texts,genres,vocab



# This function can add other features you want that help classification
# accuracy, such as bigrams, word prefixes and suffixes, etc.
# Adding Lemmatization class improve model features and remove redundant feature
class LemmaTokenizer:
     def __init__(self):
         self.wnl = WordNetLemmatizer()
     def __call__(self, doc):
         return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

def createInterestingFeatures(corpus):
  #Your code here
  data = pd.DataFrame(corpus)
  vectorizer = CountVectorizer(
      decode_error = 'ignore',
      token_pattern = '\w*[a-zA-Z]\w*',
      analyzer = 'word',
      min_df = 3,
      binary = 'True',
      tokenizer = LemmaTokenizer()
  )
  texts = vectorizer.fit_transform(data.text)
  vocab = vectorizer.get_feature_names()
  genres = data['genre']
  return texts,genres,vocab

In [73]:
#given a numpy matrix representation of the features for the training set, the 
# vector of true classes for each example, and the vocabulary as described 
# above, this computes the accuracy of the model using leave one out cross 
# validation and reports the most indicative features for each class
def evaluateModel(X,y,vocab,penalty="l1"):
  #create and fit the model
  model = LogisticRegression(penalty=penalty,solver="liblinear")
  results = cross_validate(model,X,y,cv=LeaveOneOut())
  
  #determine the average accuracy
  scores = results["test_score"]
  avg_score = sum(scores)/len(scores)
  
  #determine the most informative features
  # this requires us to fit the model to everything, because we need a
  # single model to draw coefficients from, rather than 26
  model.fit(X,y)
  neg_class_prob_sorted = model.coef_[0, :].argsort()
  pos_class_prob_sorted = (-model.coef_[0, :]).argsort()

  termsToTake = 20
  pos_indicators = [vocab[i] for i in neg_class_prob_sorted[:termsToTake]]
  neg_indicators = [vocab[i] for i in pos_class_prob_sorted[:termsToTake]]

  return avg_score,pos_indicators,neg_indicators

def runEvaluation(X,y,vocab):
  print("----------L1 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l1")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  #this call will fit a model with L2 normalization
  print("----------L2 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l2")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  

In [74]:
corpus = readShakespeare()

Run the following to train and evaluate two models with basic features:

In [75]:
X,y,vocab = createBasicFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.615385
The most informative terms for pos are: ['you', 'helena', 'duke', 'prospero', 'i', 'sir', 'leontes', 'a', 'private', 'preserving', 'preservers', 'preserver', 'preserved', 'preserve', 'preservative', 'president', 'preservation', 'presents', 'presentment', 'presently']
The most informative terms for neg are: ['him', 's', 'iago', 'imogen', 'brutus', 'o', 'lear', 'ham', 'and', 'rom', 'the', 'to', 'press', 'president', 'preserving', 'preservers', 'preserved', 'pressed', 'preserve', 'preservative']
----------L2 Norm-----------
The model's average accuracy is 0.769231
The most informative terms for pos are: ['i', 'you', 'duke', 'prospero', 'a', 'helena', 'your', 'antonio', 'sir', 'leontes', 'hermia', 'for', 'lysander', 'ariel', 'sebastian', 'demetrius', 'camillo', 'stephano', 'me', 'parolles']
The most informative terms for neg are: ['iago', 'othello', 's', 'him', 'imogen', 'what', 'lear', 'brutus', 'his', 'cassio', 'o', 'h

Run the following to train and evaluate two models with features that are interesting for distinguishing comedy and tragedy:

In [76]:
X,y,vocab = createInterestingFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.769231
The most informative terms for pos are: ['duke', 'tarry', 'jest', 'welkin', 'credit', 'studied', 'pretty', 'prettily', 'prettiest', 'pretend', 'pretence', 'presume', 'press', 'preserver', 'preserved', 'preserve', 'presenting', 'presently', 'prevail', 'presented']
The most informative terms for neg are: ['slain', 'closet', 'pledge', 'mutiny', 'camp', 'bleeding', 'purge', 'prettily', 'prettiest', 'pretend', 'pretence', 'presume', 'press', 'preserver', 'preserved', 'preserve', 'preserv', 'presenting', 'pretty', 'presented']
----------L2 Norm-----------
The model's average accuracy is 0.923077
The most informative terms for pos are: ['duke', 'apparel', 'jest', 'garment', 'commodity', 'reasonable', 'velvet', 'studied', 'mutton', 'horn', 'coat', 'credit', 'marvellous', 'lower', 'impossible', 'clock', 'shallow', 'imagination', 'tarry', 'lying']
The most informative terms for neg are: ['slain', 'camp', 'fierce', 'closet', 'e

## For understanding the context and differentiating the examples in genres like comedy and tragedies. I basically infer that comedy genre would contain the features that are positive in nature, that is something that can express happiness whereas in tragedies the features could be of negative context as we see there are few features like bleeding etc.


1.   In the **BasicFeatures**, I have used the similar function that was used in the previous part and we can clearly observe from the most informative terms that we can't infer anything about the genre from them, the most informative feature for comedy is 'you' which doesn't make any sense contextually. Also, another observation is there are similar features like 'preserving', 'preserved' in both classes. Therefore, our model is performing not well and even features are not informative. The reported accuracy for the evalutation model, **Logistic Regression with L1 norm is 61.53% and L2 norm is 76.9%**
2.   In the **createInterestingFeatures**, I have added few more parameters in the COuntVectorizer() so as to make the model more informative and contextually represent the genre. I have added **min_df** parameter so as to remove the unimportant tokens from the list of features. For example min_df = 3, wil lvremove all the tokens from the sparse matrix whose count is <=3, and this particularly improved the model accuracy for L2 norm to a great extend as the probabilities will change significantly. Also, to remove the redundant features I have added Lemmatization as well, which significantly improved the informative feature list. We can observe that informative terms now contextually understand the terms comedies and tragedies as we see more off words which denotes "Happiness" for the genre "comedies" and negative words like ['bleeding','destruction','mutiny'] for genre "tragedies". The reported accuracy for the evalutation model, **Logistic Regression with L1 norm is 76.92% and L2 norm is 92.30%**

